# A parser for NOAA bulletins

In [1]:
import feedparser

In [2]:
import numpy as np
import pandas as pd

In [3]:
import urllib

In [4]:
import re

In [8]:
from Storms.utils import readxml

In [9]:
url='http://www.nhc.noaa.gov/index-at.xml'

In [10]:
url='http://www.nhc.noaa.gov/index-ep.xml'

In [11]:
atl=feedparser.parse(url)

In [16]:
data=atl['entries']
try:
    data[0]['summary']
except:
    print data

[]


In [ ]:
print readxml(url,'lxml')

In [ ]:
data

In [ ]:
buls = len(data)

In [ ]:
for b in range(buls):
    print (data[b].summary)

## retrieve from source

In [18]:
url = 'https://metoc.ndbc.noaa.gov/RSSFeeds-portlet/img/jtwc/jtwc.rss'

In [ ]:
print readxml(url,'lxml')

In [ ]:
jtwc=feedparser.parse(url) # parse the webpage

In [ ]:
# collect the summeries (one for each alert present in the webpage)
data=jtwc['entries']
for k in range(len(data)):
    if 'tcw' in data[k].summary : txt = (data[k].summary)

In [ ]:
elp = [re.sub('<[^<]+>', "", elem) for elem in txt.split('Tropical')] # we split the text to get the references for the alerts

In [ ]:
hur = [elem.strip().split('\n')[0] for elem in elp] # the names of the TCs are...

In [ ]:
hur

In [ ]:
elp = [re.sub('<[^<]+>', "", elem) for elem in txt.split('href=')] # we now split differently to get the urls for the bulletins

In [ ]:
bul = [elem for elem in elp if 'tcw' in elem] # collect the urls 

bul_ =  [elem.strip().split()[0] for elem in bul]

bul_

In [ ]:
bul_ = [ elem.encode('utf8').replace('"', '') for elem in bul_] # get rid of unicode characters

In [ ]:
det = urllib.urlopen(bul_[1]).read() # download and parse the bulletin file

In [ ]:
print det

In [ ]:
#select the first lines where all the info is (conviniently) stored
data =[line.strip() for line in det.splitlines() if ('T' is line.strip()[0]) & ('QD' in line.strip()[-2:])]

In [ ]:
data = [re.sub(' ',',',elem) for elem in data] # replace space with commas as delimiter

In [ ]:
data = [v.split(',') for v in data] #split to array 

In [ ]:
df = pd.DataFrame(data) # create a dataframe

In [ ]:
df.head() #check

In [ ]:
# drop the text columns [NE,QD,....]
#df = df.drop(df.columns[[6,7,9,10,12,13,15,16,19,20,22,23,25,26,28,29,32,33,35,36,38,39,41,42]], axis=1) 
for ref in ['NE','NW','SW','SE','QD']:
df = df.loc[:, (df != [ref]).all(axis=0)] 

In [ ]:
df = df.set_index(df.columns[0]) # set time as index

In [ ]:
df.head() #check

In [ ]:
df.columns=np.arange(df.shape[1]) # rename the columns

In [ ]:
df.head()

In [ ]:
#move the values to appropiate place based on the wind radii in order to force [64,50,34].

for i in range(df.shape[0]):
    if df.iloc[i,3]=='R034' : 
        df.iloc[i,13:] = df.iloc[i,3:8].values
        df.iloc[i,3:8] = None
    elif df.iloc[i,3]=='R050' :
        df.iloc[i,8:] = df.iloc[i,3:13].values
        df.iloc[i,3:8] = None


In [ ]:
df #check

In [ ]:
df = df.drop(df.columns[[3,8,13]], axis=1) # drop text columns [R064,R050,R034]

In [ ]:
#set columns
df.columns=['lat','lon','vmax','64ne','64se','64sw','64nw','50ne','50se','50sw','50nw','34ne','34se','34sw','34nw']

In [ ]:
df #check

In [ ]:
tidx = [np.float(elem[1:]) for elem in df.index.values] #convert time from string to float 

In [ ]:
df.index=tidx # set float time as index

In [ ]:
df # check